In [6]:
import boto3
import pandas as pd
from io import StringIO

In [7]:
AWS_ACCESS_KEY=""
AWS_SECRET_KEY=""
AWS_REGION="ap-south-1"
SCHEMA_NAME="covid-19"
S3_STAGING_DIR="s3://indeevar-test-bucket/output/"
S3_BUCKET_NAME="indeevar-test-bucket"
S3_OUTPUT_DIRECTORY="output"

In [8]:
#Connecting to Athena
athena_client=boto3.client("athena",aws_access_key_id=AWS_ACCESS_KEY,aws_secret_access_key=AWS_SECRET_KEY,region_name=AWS_REGION)

In [10]:
Dict={}
def download_and_load_query_result(client:boto3.client,query_response: Dict )->pd.DataFrame:
    while True:
        try:
            #Loads only first 1000 rows
            client.get_query_execution(QueryExecutionId=query_response["QueryExecutionId"])
            break
        except Exception as err:
            if "not yet finished" in str(err):
                time.sleep(0.001)
            else:
                raise err
    temp_file_location: str="athena_query_results.csv"
    s3_client=boto3.client("s3",aws_access_key_id=AWS_ACCESS_KEY,aws_secret_access_key=AWS_SECRET_KEY,region_name=AWS_REGION)
    s3_client.download_file(S3_BUCKET_NAME,f"{S3_OUTPUT_DIRECTORY}/{query_response['QueryExecutionId']}.csv",temp_file_location)
    return pd.read_csv(temp_file_location)
        

In [11]:
response=athena_client.start_query_execution(QueryString="SELECT * FROM enigma_jhud",QueryExecutionContext={"Database":SCHEMA_NAME},ResultConfiguration={
    "OutputLocation":S3_STAGING_DIR,
    "EncryptionConfiguration":{"EncryptionOption":"SSE_S3"}
},)
#df_data=download_and_load_query_result(athena_client,response)


In [12]:
response


{'QueryExecutionId': '76dd8d6f-0771-49cf-b3ce-e0971908a332',
 'ResponseMetadata': {'RequestId': '4bc4b961-15ce-4f72-8250-a95f0836b4a7',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sat, 19 Aug 2023 01:53:51 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '59',
   'connection': 'keep-alive',
   'x-amzn-requestid': '4bc4b961-15ce-4f72-8250-a95f0836b4a7'},
  'RetryAttempts': 0}}

In [13]:
enigma_jhud=download_and_load_query_result(athena_client,response)

In [14]:
enigma_jhud


,fips,admin2,province_state,country_region,last_update,latitude,longitude,confirmed,deaths,recovered,active,combined_key,partition_0
0,NaN,NaN,Anhui,China,2020-01-22T17:00:00,31.826,117.226,1.0,NaN,NaN,NaN,"""Anhui",csv
1,NaN,NaN,Beijing,China,2020-01-22T17:00:00,40.182,116.414,14.0,NaN,NaN,NaN,"""Beijing",csv
2,NaN,NaN,Chongqing,China,2020-01-22T17:00:00,30.057,107.874,6.0,NaN,NaN,NaN,"""Chongqing",csv
3,NaN,NaN,Fujian,China,2020-01-22T17:00:00,26.079,117.987,1.0,NaN,NaN,NaN,"""Fujian",csv
4,NaN,NaN,Gansu,China,2020-01-22T17:00:00,36.061,103.834,NaN,NaN,NaN,NaN,"""Gansu",csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...
46138,12015.0,Charlotte,Florida,US,2020-04-04T23:34:00,26.895,-81.910,46.0,1.0,0.0,0.0,"""Charlotte",csv
46139,51037.0,Charlotte,Virginia,US,2020-04-04T23:34:00,37.019,-78.663,1.0,0.0,0.0,0.0,"""Charlotte",csv
46140,51540.0,Charlottesville,Virginia,US,2020-04-04T23:34:00,38.035,-78.486,21.0,1.0,0.0,0.0,"""Charlottesville",csv
46141,13049.0,Charlton,Georgia,US,2020-04-04T23:34:00,30.784,-82.140,1.0,0.0,0.0,0.0,"""Charlton",csv


In [15]:
enigma_jhud.to_csv('enigma_jhud')

In [17]:
#nytimes_data_in_usa_us_county
response=athena_client.start_query_execution(QueryString="SELECT * FROM nytimes_data_in_usa_us_county",QueryExecutionContext={"Database":SCHEMA_NAME},ResultConfiguration={
    "OutputLocation":S3_STAGING_DIR,
    "EncryptionConfiguration":{"EncryptionOption":"SSE_S3"}
},)
# response=athena_client.start_query_execution(QueryString="SELECT * FROM rearc-covid-19-testing-data_states_dailystates_daily",QueryExecutionContext={"Database":SCHEMA_NAME},ResultConfiguration={
#     "OutputLocation":S3_STAGING_DIR,
#     "EncryptionConfiguration":{"EncryptionOption":"SSE_S3"}
# },)
nytimes_data_in_usa_us_county=download_and_load_query_result(athena_client,response)

In [18]:
#nytimes_data_in_usa_us_states
response2=athena_client.start_query_execution(QueryString="SELECT * FROM nytimes_data_in_usa_us_states",QueryExecutionContext={"Database":SCHEMA_NAME},ResultConfiguration={
    "OutputLocation":S3_STAGING_DIR,
    "EncryptionConfiguration":{"EncryptionOption":"SSE_S3"}
},)
nytimes_data_in_usa_us_states=download_and_load_query_result(athena_client,response2)

In [19]:
#rearc-covid-19-testing-data_states_dailystates_daily
response3=athena_client.start_query_execution(QueryString="SELECT * FROM rearc-covid-19-testing-data_states_dailystates_daily",QueryExecutionContext={"Database":SCHEMA_NAME},ResultConfiguration={
    "OutputLocation":S3_STAGING_DIR,
    "EncryptionConfiguration":{"EncryptionOption":"SSE_S3"}
},)
rearc-covid_19_testing-data_states_dailystates_daily=download_and_load_query_result(athena_client,response3)

In [20]:
#rearc-covid-19-testing-data_us_dailyus_daily
response4=athena_client.start_query_execution(QueryString="SELECT * FROM rearc-covid-19-testing-data_us_dailyus_daily",QueryExecutionContext={"Database":SCHEMA_NAME},ResultConfiguration={
    "OutputLocation":S3_STAGING_DIR,
    "EncryptionConfiguration":{"EncryptionOption":"SSE_S3"}
},)
rearc_covid_19_testing-data_us_dailyus_daily=download_and_load_query_result(athena_client,response4)

In [21]:
#rearc-covid-19-testing-data_us_total_latest
response5=athena_client.start_query_execution(QueryString="SELECT * FROM rearc-covid-19-testing-data_us_total_latest",QueryExecutionContext={"Database":SCHEMA_NAME},ResultConfiguration={
    "OutputLocation":S3_STAGING_DIR,
    "EncryptionConfiguration":{"EncryptionOption":"SSE_S3"}
},)
rearc_covid_19_testing_data_us_total_latest=download_and_load_query_result(athena_client,response5)

In [22]:
#bedsdummy
response6=athena_client.start_query_execution(QueryString="SELECT * FROM bedsdummy",QueryExecutionContext={"Database":SCHEMA_NAME},ResultConfiguration={
    "OutputLocation":S3_STAGING_DIR,
    "EncryptionConfiguration":{"EncryptionOption":"SSE_S3"}
},)
bedsdummy=download_and_load_query_result(athena_client,response6)

In [23]:
#static_datacountrycode
response7=athena_client.start_query_execution(QueryString="SELECT * FROM static_datacountrycode",QueryExecutionContext={"Database":SCHEMA_NAME},ResultConfiguration={
    "OutputLocation":S3_STAGING_DIR,
    "EncryptionConfiguration":{"EncryptionOption":"SSE_S3"}
},)
static_datacountrycode=download_and_load_query_result(athena_client,response7)

In [24]:
#static_datacountypopulation
response8=athena_client.start_query_execution(QueryString="SELECT * FROM static_datacountypopulation",QueryExecutionContext={"Database":SCHEMA_NAME},ResultConfiguration={
    "OutputLocation":S3_STAGING_DIR,
    "EncryptionConfiguration":{"EncryptionOption":"SSE_S3"}
},)
static_datacountypopulation=download_and_load_query_result(athena_client,response8)

In [25]:
#static_datastate_abv
response9=athena_client.start_query_execution(QueryString="SELECT * FROM static_datastate_abv",QueryExecutionContext={"Database":SCHEMA_NAME},ResultConfiguration={
    "OutputLocation":S3_STAGING_DIR,
    "EncryptionConfiguration":{"EncryptionOption":"SSE_S3"}
},)
static_datastate_abv=download_and_load_query_result(athena_client,response9)

In [14]:
nytimes_data_in_usa_us_county=pd.read_csv("C:\Users\INDEEVAR\OneDrive\Desktop\Covid-19\enigma-nytimes-data-in-usa\csv\us_county\us_county.csv")
nytimes_data_in_usa_us_states
rearc_covid_19_testing_data_states_dailystates_daily
rearc-covid-19-testing-data_us_dailyus_daily
rearc-covid-19-testing-data_us_total_latest
bedsdummy
static_datacountrycode
static_datacountypopulation
static_datastate_abv


NameError: name 'static_datastate_abv' is not defined

In [16]:
nytimes_data_in_usa_us_county=pd.read_csv("C:\\Users\\INDEEVAR\\OneDrive\\Desktop\\Covid-19\\enigma-nytimes-data-in-usa\\csv\\us_county\\us_county.csv")

In [28]:
nytimes_data_in_usa_us_country

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1.0,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1.0,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1.0,0.0
3,2020-01-24,Cook,Illinois,17031.0,1.0,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1.0,0.0
...,...,...,...,...,...,...
26895,2020-04-02,Lincoln,Kentucky,21137.0,1.0,0.0
26896,2020-04-02,Logan,Kentucky,21141.0,2.0,0.0
26897,2020-04-02,Lyon,Kentucky,21143.0,2.0,0.0
26898,2020-04-02,Madison,Kentucky,21151.0,12.0,0.0


In [17]:
nytimes_data_in_usa_us_states=pd.read_csv("C:\\Users\\INDEEVAR\\OneDrive\\Desktop\\Covid-19\\enigma-nytimes-data-in-usa\\csv\\us_states\\us_states.csv")

In [30]:
nytimes_data_in_usa_us_states

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0
...,...,...,...,...,...
3749,2020-05-09,Virginia,51,23196,827
3750,2020-05-09,Washington,53,17763,925
3751,2020-05-09,West Virginia,54,1347,53
3752,2020-05-09,Wisconsin,55,9939,398


In [18]:
rearc_covid_19_testing_data_states_dailystates_daily=pd.read_csv("C:\\Users\\INDEEVAR\\OneDrive\\Desktop\\Covid-19\\rearc-covid-19-testing-data\\csv\\states_daily\\states_daily.csv")

In [19]:
rearc_covid_19_testing_data_us_dailyus_daily=pd.read_csv("C:\\Users\\INDEEVAR\\OneDrive\\Desktop\\Covid-19\\rearc-covid-19-testing-data\\csv\\us_daily\\us_daily.csv")

In [20]:
rearc_covid_19_testing_data_us_total_latest=pd.read_csv("C:\\Users\\INDEEVAR\\OneDrive\\Desktop\\Covid-19\\rearc-covid-19-testing-data\\csv\\us-total-latest\\us.csv")

In [21]:
usa_hospital_beds=pd.read_csv("C:\\Users\\INDEEVAR\\OneDrive\\Desktop\\Covid-19\\rearc-usa-hospital-beds\\usa-hospital-beds.csv")

In [22]:
static_datacountrycode=pd.read_csv("C:\\Users\\INDEEVAR\\OneDrive\\Desktop\\Covid-19\\static-datasets\\csv\\countrycode\\CountryCodeQS.csv")

In [23]:
static_datacountypopulation=pd.read_csv("C:\\Users\\INDEEVAR\\OneDrive\\Desktop\\Covid-19\\static-datasets\\csv\\CountyPopulation\\County_Population.csv")

In [24]:
static_datastate_abv=pd.read_csv("C:\\Users\\INDEEVAR\\OneDrive\\Desktop\\Covid-19\\static-datasets\\csv\\state-abv\\states_abv.csv")

In [25]:
factCovid_1=enigma_jhud[['fips','province_state','country_region','confirmed','deaths','recovered','active']]
factCovid_2=rearc_covid_19_testing_data_states_dailystates_daily[['fips','date','positive','negative','hospitalizedCurrently','hospitalized','hospitalizedDischarged']]
factCovid=pd.merge(factCovid_1,factCovid_2,on='fips',how='inner')

In [26]:
dimRegion_1=enigma_jhud[['fips','province_state','country_region','latitude','longitude']]
dimRegion_2=nytimes_data_in_usa_us_county[['fips','county','state']]
dimRegion=pd.merge(dimRegion_1,dimRegion_2,on='fips',how='inner')

In [29]:
usa_hospital_beds=usa_hospital_beds[['fips','state_name','latitude','longtitude','hq_address','hospital_name','hospital_type','hq_city','hq_state']]

In [28]:
usa_hospital_beds.columns=usa_hospital_beds.columns.str.lower()
usa_hospital_beds

,objectid,hospital_name,hospital_type,hq_address,hq_address1,hq_city,hq_state,hq_zip_code,county_name,state_name,...,num_licensed_beds,num_staffed_beds,num_icu_beds,adult_icu_beds,pedi_icu_beds,bed_utilization,avg_ventilator_usage,potential_increase_in_bed_capac,latitude,longtitude
0,1,Phoenix VA Health Care System (AKA Carl T Hayd...,VA Hospital,650 E Indian School Rd,NaN,Phoenix,AZ,85012,Maricopa,Arizona,...,129.0,129.0,0,0,NaN,NaN,0.0,0,33.495498,-112.066157
1,2,Southern Arizona VA Health Care System,VA Hospital,3601 S 6th Ave,NaN,Tucson,AZ,85723,Pima,Arizona,...,295.0,295.0,2,2,NaN,NaN,2.0,0,32.181263,-110.965885
2,3,VA Central California Health Care System,VA Hospital,2615 E Clinton Ave,NaN,Fresno,CA,93703,Fresno,California,...,57.0,57.0,2,2,NaN,NaN,2.0,0,36.773324,-119.779742
3,4,VA Connecticut Healthcare System - West Haven ...,VA Hospital,950 Campbell Ave,NaN,West Haven,CT,6516,New Haven,Connecticut,...,216.0,216.0,1,1,NaN,NaN,2.0,0,41.284400,-72.957610
4,5,Wilmington VA Medical Center,VA Hospital,1601 Kirkwood Hwy,NaN,Wilmington,DE,19805,New Castle,Delaware,...,60.0,60.0,0,0,NaN,NaN,1.0,0,39.740206,-75.606532
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6632,6633,University of Iowa Health Network Rehabilitati...,Rehabilitation Hospital,2450 Coral Ct,NaN,Coralville,IA,52241,Johnson,Iowa,...,NaN,NaN,0,0,NaN,NaN,NaN,0,41.681410,-91.575728
6633,6634,Northwest Medical Center Sahuarita,Short Term Acute Care Hospital,16260 S Rancho Sahuarita Blvd,NaN,Sahuarita,AZ,85629,Pima,Arizona,...,NaN,NaN,0,0,NaN,NaN,NaN,0,31.955757,-110.984489
6634,6635,The Heights Hospital,Short Term Acute Care Hospital,1917 Ashland St,NaN,Houston,TX,77008,Harris,Texas,...,NaN,NaN,0,0,NaN,NaN,NaN,0,29.803581,-95.404182
6635,6636,Cobalt Rehabilitation Hospital Clarksville,Rehabilitation Hospital,2101 Broadway St,NaN,Clarksville,IN,47129,Clark,Indiana,...,NaN,NaN,0,0,NaN,NaN,NaN,0,38.329353,-85.765610


In [30]:
dim_hospital_bed=usa_hospital_beds[['fips','state_name','latitude','longtitude','hq_address','hospital_name','hospital_type','hq_city','hq_state']]

In [31]:
dimDate=rearc_covid_19_testing_data_states_dailystates_daily[['fips','date']]

In [32]:
dimDate['date']=pd.to_datetime(dimDate['date'],format='%Y%m%d')


<ipython-input-32-418e4c094b7d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dimDate['date']=pd.to_datetime(dimDate['date'],format='%Y%m%d')


In [33]:
dimDate.head()

,fips,date
0,2.0,2021-03-07
1,1.0,2021-03-07
2,5.0,2021-03-07
3,60.0,2021-03-07
4,4.0,2021-03-07


In [34]:
dimDate['year']=dimDate['date'].dt.year
dimDate['month']=dimDate['date'].dt.month
dimDate['day_of_week']=dimDate['date'].dt.dayofweek

<ipython-input-34-7b60d7ed0f61>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dimDate['year']=dimDate['date'].dt.year
<ipython-input-34-7b60d7ed0f61>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dimDate['month']=dimDate['date'].dt.month
<ipython-input-34-7b60d7ed0f61>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u

In [35]:
dimDate.head()

,fips,date,year,month,day_of_week
0,2.0,2021-03-07,2021,3,6
1,1.0,2021-03-07,2021,3,6
2,5.0,2021-03-07,2021,3,6
3,60.0,2021-03-07,2021,3,6
4,4.0,2021-03-07,2021,3,6


In [36]:
bucket="indeevar-covid-de-project"

In [84]:
#Stroing data into bucket
csv_buffer=StringIO()
factCovid.to_csv(csv_buffer)
s3_resource=boto3.resource('s3')
s3_resource.Object(bucket,'output/factCovid.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'CBF5RNHBCGJEN966',
  'HostId': 'UxfoNYypbwxSlb/cg8QEIlFgO0cjc1Crk9DbWjXb5sOgqJaRg1t0LWJuHVjSzTTlLI2+MF9KwoRgjJK8oWOp5w==',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'UxfoNYypbwxSlb/cg8QEIlFgO0cjc1Crk9DbWjXb5sOgqJaRg1t0LWJuHVjSzTTlLI2+MF9KwoRgjJK8oWOp5w==',
   'x-amz-request-id': 'CBF5RNHBCGJEN966',
   'date': 'Fri, 18 Aug 2023 16:53:24 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"30ee9d25d76581fcb05366dd59ae209e"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 1},
 'ETag': '"30ee9d25d76581fcb05366dd59ae209e"',
 'ServerSideEncryption': 'AES256'}

In [86]:
# csv_buffer.getvalue()

In [88]:
csv_buffer=StringIO()
dimRegion.to_csv(csv_buffer)
s3_resource=boto3.resource('s3')
s3_resource.Object(bucket,'output/dimRegion.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'N0PG9M7AECM30P7C',
  'HostId': 'LZEscIwHdMDDlXLgSHyHgmjpJTrqUtm6if+C66N49qx6/xXrrt7HqwF061S8m7Ur/5tMZWiczRQ=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'LZEscIwHdMDDlXLgSHyHgmjpJTrqUtm6if+C66N49qx6/xXrrt7HqwF061S8m7Ur/5tMZWiczRQ=',
   'x-amz-request-id': 'N0PG9M7AECM30P7C',
   'date': 'Fri, 18 Aug 2023 16:56:16 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"5746d49ce36b25fb33fc9d9db767f525"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 1},
 'ETag': '"5746d49ce36b25fb33fc9d9db767f525"',
 'ServerSideEncryption': 'AES256'}

In [90]:
csv_buffer=StringIO()
usa_hospital_beds.to_csv(csv_buffer)
s3_resource=boto3.resource('s3')
s3_resource.Object(bucket,'output/usa_hospital_beds.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': '021D1NVF9VNCQTA5',
  'HostId': '8FEKVDGfdGFRhgSt09dw3XkFWDSQl6O8bXFapJlVYu+hVM8vxq7/4IhIIlbHN9cDShOCbIiury0=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '8FEKVDGfdGFRhgSt09dw3XkFWDSQl6O8bXFapJlVYu+hVM8vxq7/4IhIIlbHN9cDShOCbIiury0=',
   'x-amz-request-id': '021D1NVF9VNCQTA5',
   'date': 'Fri, 18 Aug 2023 16:59:18 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"b1786e0c3a1dae6e76f7e5396dfde464"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 1},
 'ETag': '"b1786e0c3a1dae6e76f7e5396dfde464"',
 'ServerSideEncryption': 'AES256'}

In [91]:
csv_buffer=StringIO()
dimDate.to_csv(csv_buffer)
s3_resource=boto3.resource('s3')
s3_resource.Object(bucket,'output/dimDate.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'CN6M9AJA31RWZ8TX',
  'HostId': 'wC9KUMsyKYL+LzwaqdJ20003furhOd6yASvc5e6fxIPNN+eulVJbVd+X9jr7ZPeBeNmoQIdNqPs=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'wC9KUMsyKYL+LzwaqdJ20003furhOd6yASvc5e6fxIPNN+eulVJbVd+X9jr7ZPeBeNmoQIdNqPs=',
   'x-amz-request-id': 'CN6M9AJA31RWZ8TX',
   'date': 'Fri, 18 Aug 2023 17:00:04 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"19eb0b77e7f7441c686829bc3fd1a906"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 1},
 'ETag': '"19eb0b77e7f7441c686829bc3fd1a906"',
 'ServerSideEncryption': 'AES256'}

In [37]:
dimDatesql=pd.io.sql.get_schema(dimDate.reset_index(),'dimDate')
print(''.join(dimDatesql))

CREATE TABLE "dimDate" (
"index" INTEGER,
  "fips" REAL,
  "date" TIMESTAMP,
  "year" INTEGER,
  "month" INTEGER,
  "day_of_week" INTEGER
)


In [38]:
factCovidsql=pd.io.sql.get_schema(factCovid.reset_index(),'factCovid')
print(''.join(factCovidsql))

CREATE TABLE "factCovid" (
"index" INTEGER,
  "fips" REAL,
  "province_state" TEXT,
  "country_region" TEXT,
  "confirmed" REAL,
  "deaths" REAL,
  "recovered" REAL,
  "active" REAL,
  "date" INTEGER,
  "positive" INTEGER,
  "negative" REAL,
  "hospitalizedCurrently" REAL,
  "hospitalized" REAL,
  "hospitalizedDischarged" REAL
)


In [39]:
dimRegionsql=pd.io.sql.get_schema(dimRegion.reset_index(),'dimRegion')
print(''.join(dimRegionsql))

CREATE TABLE "dimRegion" (
"index" INTEGER,
  "fips" REAL,
  "province_state" TEXT,
  "country_region" TEXT,
  "latitude" REAL,
  "longitude" REAL,
  "county" TEXT,
  "state" TEXT
)


In [40]:
dimhospitalsql=pd.io.sql.get_schema(usa_hospital_beds.reset_index(),'usa_hospital_beds')
print(''.join(dimhospitalsql))

CREATE TABLE "usa_hospital_beds" (
"index" INTEGER,
  "fips" REAL,
  "state_name" TEXT,
  "latitude" REAL,
  "longtitude" REAL,
  "hq_address" TEXT,
  "hospital_name" TEXT,
  "hospital_type" TEXT,
  "hq_city" TEXT,
  "hq_state" TEXT
)


In [48]:
!pip install redshift-connector --user

  Using cached redshift_connector-2.0.913-py3-none-any.whl (114 kB)
  Using cached scramp-1.4.4-py3-none-any.whl (13 kB)
  Using cached lxml-4.9.3-cp38-cp38-win_amd64.whl (3.9 MB)


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



  Using cached asn1crypto-1.5.1-py2.py3-none-any.whl (105 kB)


In [55]:
conda install -c conda-forge redshift_connector

^C

Note: you may need to restart the kernel to use updated packages.


In [4]:
# pip list

In [41]:
import redshift_connector

In [43]:
conn=redshift_connector.connect(host='',database='dev',user="awsuser",password="")

InterfaceError: ('communication error', gaierror(11001, 'getaddrinfo failed'))

In [ ]:
enigma_jhud.to_csv('enigma_jhud')

In [45]:
factCovid.to_csv('factCovid.csv')


In [46]:
dimRegion.to_csv('dimRegion.csv')


In [47]:
usa_hospital_beds.to_csv('dimHospital.csv')

In [48]:
dimDate.to_csv('dimDate.csv')